# LOAD ALL IMAGES

In [4]:
from tqdm import tqdm
from torchvision import transforms as trn
import numpy as np
import os
from PIL import Image

images_dir = os.path.join('tutorial_images')
images_list = os.listdir(images_dir)
images_list.sort()
print('Images:')
for i, img in enumerate(images_list):
    print(str(i+1) + ': ' + img)
    
    
images = []
for img in tqdm(images_list):
    img_dir = os.path.join(images_dir, img)
    img = Image.open(img_dir).convert('RGB')
    # Center crop the images to square format, and resize them
    transform = trn.Compose([
        trn.CenterCrop(min(img.size)),
        trn.Resize((227,227))
    ])
    img = transform(img)
    img = np.asarray(img)
    img = img.transpose(2,0,1)
    images.append(img)
images = np.asarray(images)

# Print the images dimensions
print('\n\nImages shape:')
print(images.shape)
print('(Batch size × 3 RGB Channels x Width x Height)')

Images:
1: 000000000139.jpg
2: 000000000285.jpg
3: 000000000632.jpg
4: 000000000724.jpg
5: 000000000776.jpg
6: 000000000785.jpg
7: 000000000802.jpg
8: 000000000872.jpg
9: 000000000885.jpg
10: 000000001000.jpg
11: 000000001268.jpg
12: 000000001296.jpg
13: 000000001353.jpg
14: 000000001425.jpg
15: 000000001490.jpg
16: 000000001503.jpg
17: 000000001532.jpg
18: 000000001584.jpg
19: 000000001675.jpg
20: 000000001761.jpg
21: 000000001818.jpg
22: 000000001993.jpg
23: 000000002006.jpg
24: 000000002149.jpg
25: 000000002153.jpg
26: 000000002157.jpg
27: 000000002261.jpg
28: 000000002299.jpg
29: 000000002431.jpg
30: 000000002473.jpg
31: 000000002532.jpg
32: 000000002587.jpg
33: 000000002592.jpg
34: 000000002685.jpg
35: 000000002923.jpg
36: 000000003156.jpg
37: 000000003255.jpg
38: 000000003501.jpg
39: 000000003553.jpg
40: 000000003661.jpg
41: 000000003845.jpg
42: 000000003934.jpg
43: 000000004134.jpg
44: 000000004395.jpg
45: 000000004495.jpg
46: 000000004765.jpg
47: 000000004795.jpg
48: 0000000050

100%|██████████| 100/100 [00:00<00:00, 320.13it/s]



Images shape:
(100, 3, 227, 227)
(Batch size × 3 RGB Channels x Width x Height)


---

# Testing FMRI Model

In [1]:
from new_nest import NEST
nest = NEST(nest_dir="./")

In [2]:
# List all available models and their versions
available_models = nest.list_models()
print(f"Available models: {available_models}")

# See what modalities are available
modalities = nest.which_modalities()
print(f"Available modalities: {modalities}")

Available models: {'fmri_nsd_fwrf': ['1.0.0'], 'eeg_things_eeg_2_vit_b_32': ['1.0.0']}
Available modalities: ['eeg', 'fmri']


In [3]:
# Choose a model of interest
model_id = "fmri_nsd_fwrf"  # Example model ID

# Get comprehensive model information
model_info = nest.get_model_info(model_id)


🧠 Model: fmri_nsd_fwrf
------------------------------------------------------------
Modality: fmri
Dataset: NSD
Features: x
Repeats: x
Subject level: x

📌 Supported Parameters:

• subject (int, required)
  ↳ Subject ID from the NSD dataset
  ↳ Valid values: [1, 2, 3, 4, 5, 6, 7, 8]
  ↳ Example: 1

• roi (str, required)
  ↳ Region of Interest (ROI) for voxel prediction
  ↳ Valid values: ['V1', 'V2', 'V3', 'hV4', 'EBA', 'FBA-2', 'OFA', 'FFA-1', 'FFA-2', 'PPA', 'RSC', 'OPA', 'OWFA', 'VWFA-1', 'VWFA-2', 'mfs-words', 'early', 'midventral', 'midlateral', 'midparietal', 'parietal', 'lateral', 'ventral']
  ↳ Example: V1

• nest_dir (str, optional)
  ↳ Root directory of the NEST repository (optional if default paths are set)
  ↳ Example: ./

📦 Example Usage:

from nest import NEST

nest = NEST("/path/to/nest_dir")
model = nest.get_encoding_model("fmri_nsd_fwrf", subject=1, roi='V1', nest_dir='./')

# Generate responses (assuming stimulus is a numpy array)
responses = model.generate_response(sti

In [4]:
fmri_model = nest.get_encoding_model("fmri_nsd_fwrf", subject=1, roi="V1", nest_dir="./")

Model loaded on cpu for subject 1, ROI V1


In [6]:
nest.encode(fmri_model, images)

Encoding fMRI responses:   0%|          | 0/1 [00:00<?, ?it/s, Encoded images=0, Total images=100]

Encoding fMRI responses: 100%|██████████| 1/1 [00:00<00:00,  1.22it/s, Encoded images=100, Total images=100]


array([[-0.055599  ,  0.1633554 ,  0.0943293 , ...,  0.01447097,
         0.15747474,  0.01369774],
       [ 0.01819719,  0.07435445,  0.04977052, ..., -0.01315978,
         0.10419706,  0.03225161],
       [ 0.10229094,  0.21433882,  0.13944887, ...,  0.01901415,
         0.14813517,  0.00760403],
       ...,
       [ 0.04800354, -0.14399454, -0.28155515, ..., -0.02397447,
        -0.08016404,  0.00045429],
       [ 0.04686468,  0.07014671,  0.00538955, ..., -0.02637868,
         0.10473584,  0.02725478],
       [ 0.11645083,  0.02392432, -0.04246235, ..., -0.07162039,
        -0.0095483 , -0.01467158]], dtype=float32)

In [5]:
fmri_model.get_metadata()

{'fmri': {'ncsnr': array([0.10146447, 0.30089897, 0.2398103 , ..., 0.04201289, 0.22930188,
         0.06711598]),
  'roi_mask_volume': array([[[False, False, False, ..., False, False, False],
          [False, False, False, ..., False, False, False],
          [False, False, False, ..., False, False, False],
          ...,
          [False, False, False, ..., False, False, False],
          [False, False, False, ..., False, False, False],
          [False, False, False, ..., False, False, False]],
  
         [[False, False, False, ..., False, False, False],
          [False, False, False, ..., False, False, False],
          [False, False, False, ..., False, False, False],
          ...,
          [False, False, False, ..., False, False, False],
          [False, False, False, ..., False, False, False],
          [False, False, False, ..., False, False, False]],
  
         [[False, False, False, ..., False, False, False],
          [False, False, False, ..., False, False, False],
   

# Testing EEG Model

In [1]:
from new_nest import NEST
nest = NEST(nest_dir="./")

# List all available models and their versions
available_models = nest.list_models()
print(f"Available models: {available_models}")

# See what modalities are available
modalities = nest.which_modalities()
print(f"Available modalities: {modalities}")


Available models: {'fmri_nsd_fwrf': ['1.0.0'], 'eeg_things_eeg_2_vit_b_32': ['1.0.0']}
Available modalities: ['eeg', 'fmri']


In [2]:
eeg_model = nest.get_encoding_model("eeg_things_eeg_2_vit_b_32", subject=1, nest_dir="./")


{'eeg': {'ch_names': ['Fp1', 'F3', 'F7', 'FT9', 'FC5', 'FC1', 'C3', 'T7', 'TP9', 'CP5', 'CP1', 'Pz', 'P3', 'P7', 'O1', 'Oz', 'O2', 'P4', 'P8', 'TP10', 'CP6', 'CP2', 'Cz', 'C4', 'T8', 'FT10', 'FC6', 'FC2', 'F4', 'F8', 'Fp2', 'AF7', 'AF3', 'AFz', 'F1', 'F5', 'FT7', 'FC3', 'FCz', 'C1', 'C5', 'TP7', 'CP3', 'P1', 'P5', 'PO7', 'PO3', 'POz', 'PO4', 'PO8', 'P6', 'P2', 'CPz', 'CP4', 'TP8', 'C6', 'C2', 'FC4', 'FT8', 'F6', 'F2', 'AF4', 'AF8'], 'times': array([-0.1  , -0.095, -0.09 , -0.085, -0.08 , -0.075, -0.07 , -0.065,
       -0.06 , -0.055, -0.05 , -0.045, -0.04 , -0.035, -0.03 , -0.025,
       -0.02 , -0.015, -0.01 , -0.005,  0.   ,  0.005,  0.01 ,  0.015,
        0.02 ,  0.025,  0.03 ,  0.035,  0.04 ,  0.045,  0.05 ,  0.055,
        0.06 ,  0.065,  0.07 ,  0.075,  0.08 ,  0.085,  0.09 ,  0.095,
        0.1  ,  0.105,  0.11 ,  0.115,  0.12 ,  0.125,  0.13 ,  0.135,
        0.14 ,  0.145,  0.15 ,  0.155,  0.16 ,  0.165,  0.17 ,  0.175,
        0.18 ,  0.185,  0.19 ,  0.195,  0.2  ,  0.205,  0

In [3]:
eeg_model.get_metadata()

{'eeg': {'ch_names': ['Fp1',
   'F3',
   'F7',
   'FT9',
   'FC5',
   'FC1',
   'C3',
   'T7',
   'TP9',
   'CP5',
   'CP1',
   'Pz',
   'P3',
   'P7',
   'O1',
   'Oz',
   'O2',
   'P4',
   'P8',
   'TP10',
   'CP6',
   'CP2',
   'Cz',
   'C4',
   'T8',
   'FT10',
   'FC6',
   'FC2',
   'F4',
   'F8',
   'Fp2',
   'AF7',
   'AF3',
   'AFz',
   'F1',
   'F5',
   'FT7',
   'FC3',
   'FCz',
   'C1',
   'C5',
   'TP7',
   'CP3',
   'P1',
   'P5',
   'PO7',
   'PO3',
   'POz',
   'PO4',
   'PO8',
   'P6',
   'P2',
   'CPz',
   'CP4',
   'TP8',
   'C6',
   'C2',
   'FC4',
   'FT8',
   'F6',
   'F2',
   'AF4',
   'AF8'],
  'times': array([-0.1  , -0.095, -0.09 , -0.085, -0.08 , -0.075, -0.07 , -0.065,
         -0.06 , -0.055, -0.05 , -0.045, -0.04 , -0.035, -0.03 , -0.025,
         -0.02 , -0.015, -0.01 , -0.005,  0.   ,  0.005,  0.01 ,  0.015,
          0.02 ,  0.025,  0.03 ,  0.035,  0.04 ,  0.045,  0.05 ,  0.055,
          0.06 ,  0.065,  0.07 ,  0.075,  0.08 ,  0.085,  0.09 ,  0.095,
   

In [6]:
nest.encode(eeg_model, images,return_metadata=True)

Encoding EEG responses:   0%|          | 0/1 [00:00<?, ?it/s]

Encoding EEG responses: 100%|██████████| 1/1 [00:01<00:00,  1.74s/it, Encoded images=100, Total images=100]


(array([[[[ 1.24397352e-01,  7.15943575e-02, -3.34081054e-03, ...,
            1.98921803e-02, -5.35618514e-04,  2.01637093e-02],
          [-3.15451100e-02,  4.23475057e-02,  1.02535307e-01, ...,
            9.39431414e-02,  2.16076255e-01,  2.00244278e-01],
          [-1.09511185e-02, -2.08470270e-01, -3.59001279e-01, ...,
           -8.20529461e-02, -6.36036918e-02, -1.31917879e-01],
          ...,
          [-2.55278051e-02, -9.43882950e-03,  4.79462147e-02, ...,
            1.25528991e-01,  1.35852039e-01,  7.83592612e-02],
          [-1.39562547e-01, -5.55058941e-02,  2.38118283e-02, ...,
            9.01250839e-02,  1.67249560e-01,  8.69320780e-02],
          [-6.74541444e-02, -1.07088499e-01, -1.07773006e-01, ...,
           -1.29966959e-01, -9.95634496e-02, -8.21912885e-02]],
 
         [[ 7.96499774e-02, -9.57980528e-02, -1.29369929e-01, ...,
           -1.10400274e-01, -1.01401001e-01, -6.63073510e-02],
          [-1.31157815e-01,  1.05764205e-02,  4.61376347e-02, ...,
     